# **1. Thêm thư viện cần thiết**

In [1]:
# !pip install -U transformers

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch

2025-06-15 05:31:34.882970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749965495.073681      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749965495.127149      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# **2. Tải mô hình và dữ liệu**

In [3]:
model_name ="VietAI/vit5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Load dataset từ CSV
# Load cả train & val từ CSV
dataset = load_dataset(
    "csv", 
    data_files={
        "train": "/kaggle/input/disease-qa-data-v2/disease_QA_v2_train.csv",
        "validation": "/kaggle/input/disease-qa-data-v2/disease_QA_v2_val.csv"
    }
)

tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/904M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

**Thông tin về mô hình**

In [4]:
# =======================================================
# Bắt đầu in toàn bộ thông tin mô hình
# =======================================================

print("\n========== Thông tin tổng quát về mô hình ==========")

# Tổng số tham số
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Tổng số tham số: {total_params:,}")
print(f"Số tham số trainable: {trainable_params:,}")

# Thông tin về cấu hình mô hình
config = model.config

print(f"\nMô hình loại: {config.model_type}")
print(f"Chiều embedding (d_model): {config.d_model}")
print(f"Số encoder layers: {config.num_layers}")
print(f"Số decoder layers: {config.num_decoder_layers}")
print(f"Số attention heads (encoder): {config.num_heads}")
print(f"Kích thước intermediate FFN: {config.d_ff}")
print(f"Số chiều context tối đa (max_position_embeddings): {config.n_positions}")
print(f"Torch dtype: {model.dtype}")

# Thông tin về vocab
print(f"\nKích thước từ vựng (vocab_size): {config.vocab_size}")

# Kiểm tra embedding dimension
embed_weight = model.get_input_embeddings().weight
print(f"Chiều vector embedding: {embed_weight.shape[1]}")

# Thông tin về tokenizer
print("\n========== Thông tin tokenizer ==========")
print(f"Padding token id: {tokenizer.pad_token_id}")
print(f"BOS token id: {tokenizer.bos_token_id}")
print(f"EOS token id: {tokenizer.eos_token_id}")
print(f"Special tokens: {tokenizer.special_tokens_map}")

# In danh sách token đặc biệt
print("\nDanh sách các token đặc biệt:")
for name, token in tokenizer.special_tokens_map.items():
    print(f"- {name}: {token}")


========== Thông tin tổng quát về mô hình ==========
Tổng số tham số: 225,950,976
Số tham số trainable: 225,950,976

Mô hình loại: t5
Chiều embedding (d_model): 768
Số encoder layers: 12
Số decoder layers: 12
Số attention heads (encoder): 12
Kích thước intermediate FFN: 3072
Số chiều context tối đa (max_position_embeddings): 512
Torch dtype: torch.float32

Kích thước từ vựng (vocab_size): 36096
Chiều vector embedding: 768

========== Thông tin tokenizer ==========
Padding token id: 0
BOS token id: None
EOS token id: 1
Special tokens: {'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>',

**Thống kê dữ liệu**

In [5]:
# train_dataset = dataset['train']

# # Hàm đếm số lượng token cho 1 text
# def count_tokens(text):
#     return len(tokenizer.encode(str(text), add_special_tokens=False))

# # Đếm số token cho từng trường, mỗi dòng
# title_tokens = [count_tokens(x) for x in train_dataset['Title']]
# question_tokens = [count_tokens(x) for x in train_dataset['Question']]
# answer_tokens = [count_tokens(x) for x in train_dataset['Answer']]
# type_tokens = [count_tokens(x) for x in train_dataset['Type']]

# print("== Giá trị token lớn nhất trong mỗi trường ==")
# print(f"Max token Title: {max(title_tokens)}")
# print(f"Max token Question: {max(question_tokens)}")
# print(f"Max token Answer: {max(answer_tokens)}")
# print(f"Max token Type: {max(type_tokens)}")

# import matplotlib.pyplot as plt
# import numpy as np
# # Tính số lượng token trung bình của từng trường
# fields = ['Title', 'Question', 'Answer', 'Type']
# avg_tokens = [
#     np.mean([count_tokens(x) for x in train_dataset['Title']]),
#     np.mean([count_tokens(x) for x in train_dataset['Question']]),
#     np.mean([count_tokens(x) for x in train_dataset['Answer']]),
#     np.mean([count_tokens(x) for x in train_dataset['Type']])
# ]

# # Vẽ biểu đồ cột
# plt.figure(figsize=(8, 5))
# plt.bar(fields, avg_tokens)
# plt.ylabel('Số lượng token trung bình')
# plt.title('Biểu đồ số lượng token trung bình của từng trường')
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.show()

In [6]:
# def preprocess(examples):
#     inputs = examples['Question']
#     targets = examples['Answer']
#     model_inputs = tokenizer(inputs, max_length=69, truncation=True, padding="max_length")
#     # Tokenize labels
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(targets, max_length=200, truncation=True, padding="max_length")

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# tokenized_datasets = dataset.map(preprocess, batched=True)



# **3. Xử lý dữ liệu huấn luyện**

In [7]:
def preprocess(examples):
    inputs = examples['Question']
    targets = examples['Answer']
    model_inputs = tokenizer(inputs, max_length=69, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=187, truncation=True, padding="max_length")
    # Padding label về -100
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Preprocess cho cả train và validation
tokenized_datasets = dataset.map(preprocess, batched=True)

# Xóa cột thừa để tiết kiệm RAM, tránh lỗi Trainer (nếu dataset có các cột này)
for split in ["train", "validation"]:
    cols_to_remove = [col for col in tokenized_datasets[split].column_names if col not in ["input_ids", "attention_mask", "labels"]]
    tokenized_datasets[split] = tokenized_datasets[split].remove_columns(cols_to_remove)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Map:   0%|          | 0/20819 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2081 [00:00<?, ? examples/s]

In [8]:
# Lấy thử một sample từ train dataset
sample = tokenized_datasets["train"][0]

# In input_ids (decode về text để kiểm tra)
print("========== INPUT ==========")
input_text = tokenizer.decode(sample["input_ids"], skip_special_tokens=False)
print(input_text)

# In raw input_ids (chỉ số token)
print("\n========== INPUT IDS ==========")
print(sample["input_ids"])

# In attention mask
print("\n========== ATTENTION MASK ==========")
print(sample["attention_mask"])

# In labels (kiểm tra mask -100)
print("\n========== LABELS ==========")
print(sample["labels"])

# Đếm số lượng token bị mask -100 trong labels
masked_tokens = sum(1 for token in sample["labels"] if token == -100)
print(f"\nSố token bị mask -100 trong labels: {masked_tokens}")

# Nếu muốn decode labels (chỗ không bị mask):
decoded_labels = tokenizer.decode(
    [token if token != -100 else tokenizer.pad_token_id for token in sample["labels"]],
    skip_special_tokens=False
)
print("\n========== DECODED LABELS ==========")
print(decoded_labels)


========== INPUT ==========
Tôi đang cảm thấy u tế bào hạt (Oncocytom) khá hiếm gặp. Tôi có thể đang bị bệnh gì? </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

========== INPUT IDS ==========
[1556, 302, 1281, 663, 1636, 454, 2289, 1218, 33, 26970, 35788, 1098, 4158, 467, 35831, 1790, 2943, 1074, 35792, 1556, 71, 243, 302, 138, 897, 1080, 35933, 35783, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

========== ATTENTION MASK ==========
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

========== LABELS ==========
[4440, 71, 243, 302, 138, 897, 344, 9969, 7261, 3071, 35792, 1, -100, -100, -100, -100,

# **4. Fine-tunning mô hình**

In [9]:

training_args = TrainingArguments(
    output_dir="./Medical_QA_ViT5_check_point_100",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    num_train_epochs=150,                  
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    logging_strategy="steps",
    dataloader_num_workers=4,
    logging_steps=500,
    save_strategy="epoch",
    eval_strategy="epoch",        
    save_total_limit=1,
    report_to=[],
    fp16=True,
    seed=42,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

In [10]:
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# trainer.train()

In [11]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
trainer.train(resume_from_checkpoint="/kaggle/input/medical-qa-vit5-check-point-100/Medical_QA_ViT5_check_point_100/checkpoint-32500")


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
101,0.404100,0.119424
102,0.409500,0.117969
103,0.409500,0.114184
104,0.403700,0.111727
105,0.398100,0.109126
106,0.398100,0.105735
107,0.391900,0.104877
108,0.384200,0.101396
109,0.384200,0.099382
110,0.378100,0.098060


TrainOutput(global_step=48750, training_loss=0.11593651639498197, metrics={'train_runtime': 38651.6857, 'train_samples_per_second': 80.795, 'train_steps_per_second': 1.261, 'total_flos': 2.555048925487872e+17, 'train_loss': 0.11593651639498197, 'epoch': 149.8479262672811})

# **5. Dự đoán**

In [12]:

def answer_question(question, max_input_length=69, max_output_length=200):
    # Tokenize input
    inputs = tokenizer(
        question,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=max_input_length
    ).to(model.device)

    # Sinh ra câu trả lời
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=max_output_length,
            num_beams=4,              # beam search cho câu trả lời tốt hơn
            early_stopping=True
        )

    # Decode output
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer


In [13]:
questions = [
    "Tinh hoàn của con trai tôi không di chuyển xuống bìu sau 6 tháng tuổi. Nó có thể đang bị bệnh gì? ",
    "Con trai tôi sờ thấy khối u nổi lên ở ống bẹn. Nó có thể đang bị bệnh gì? ",
    "Áp xe não là gì?"
]

for q in questions:
    print(f"❓ {q}")
    print(f"➡️  {answer_question(q)}\n")


❓ Tinh hoàn của con trai tôi không di chuyển xuống bìu sau 6 tháng tuổi. Nó có thể đang bị bệnh gì? 
➡️  Bạn có thể đang bị bệnh Tinh Hoàn Ẩn.

❓ Con trai tôi sờ thấy khối u nổi lên ở ống bẹn. Nó có thể đang bị bệnh gì? 
➡️  Bạn có thể đang bị bệnh Tinh Hoàn Ẩn.

❓ Áp xe não là gì?
➡️  Áp xe não là một bệnh nhiễm trùng trong não, là một trường hợp cấp cứu y tế cần được tiến hành điều trị ngay lập tức.



In [14]:
# Lưu mô hình đã huấn luyện
model.save_pretrained("./kaggle/working/Medical_QA_model_ViT5_base_epoch_150")

# Lưu tokenizer (rất quan trọng để decode sau này!)
tokenizer.save_pretrained("./kaggle/working/Medical_QA_model_ViT5_base_epoch_150")


('./kaggle/working/Medical_QA_model_ViT5_base_epoch_150/tokenizer_config.json',
 './kaggle/working/Medical_QA_model_ViT5_base_epoch_150/special_tokens_map.json',
 './kaggle/working/Medical_QA_model_ViT5_base_epoch_150/spiece.model',
 './kaggle/working/Medical_QA_model_ViT5_base_epoch_150/added_tokens.json',
 './kaggle/working/Medical_QA_model_ViT5_base_epoch_150/tokenizer.json')

# **6. Tải mô hình lên để chạy dự đoán**

In [15]:
# checkpoint_path = "/kaggle/input/medical-qa-model-vit5-base-epoch-50/Medical_QA_model_ViT5_base_epoch_50"  # sửa đúng path checkpoint nếu cần
# tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
# model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)

# def answer_question(question, max_input_length=69, max_output_length=200):
#     inputs = tokenizer(
#         question,
#         return_tensors="pt",
#         truncation=True,
#         padding="max_length",
#         max_length=max_input_length
#     ).to(model.device)
#     with torch.no_grad():
#         output_ids = model.generate(
#             **inputs,
#             max_length=max_output_length,
#             num_beams=8,
#             early_stopping=True
#         )
#     answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
#     return answer



In [16]:
# questions = [
#     "Bệnh Thủy đậu là gì?"
# ]
# for q in questions:
#     print(f"❓ {q}")
#     print(f"➡️  {answer_question(q)}\n")

# **7. Đánh giá kết quả của mô hình trên tập test dựa vào các thang đo chuẩn**

In [17]:
!pip install evaluate
!pip install sacrebleu
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

In [18]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
from tqdm import tqdm
from evaluate import load as load_metric

# Load model và tokenizer từ checkpoint
# checkpoint_path = "/kaggle/input/medical-qa-model-vit5-base-epoch-50/Medical_QA_model_ViT5_base_epoch_50"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
# model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Load dữ liệu test từ file CSV
test_df = pd.read_csv("/kaggle/input/disease-qa-data-v2/disease_QA_v2_test.csv")
test_dataset = Dataset.from_pandas(test_df)

# Hàm sinh câu trả lời
def generate_answer(question, max_input_length=69, max_output_length=187):
    inputs = tokenizer(
        question,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=max_input_length
    ).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=max_output_length,
            num_beams=8,
            early_stopping=True
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Tính predict cho toàn bộ test set
predictions = []
references = []

for row in tqdm(test_dataset):
    question = row["Question"]
    ref_answer = row["Answer"]

    pred_answer = generate_answer(question)

    predictions.append(pred_answer)
    references.append(ref_answer)


100%|██████████| 1040/1040 [11:24<00:00,  1.52it/s]


In [19]:
# Load metric từ Hugging Face
bleu_metric = load_metric("sacrebleu")
rouge_metric = load_metric("rouge")

# Định dạng đúng cho sacrebleu
bleu_result = bleu_metric.compute(predictions=predictions, references=[[ref] for ref in references])
rouge_result = rouge_metric.compute(predictions=predictions, references=references, use_stemmer=True)

print(f"BLEU score: {bleu_result['score']:.2f}")
print("ROUGE scores:")
for key in ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']:
    print(f"{key}: {rouge_result[key]:.4f}")



BLEU score: 82.03
ROUGE scores:
rouge1: 0.9493
rouge2: 0.9242
rougeL: 0.9359
rougeLsum: 0.9361
